In [1]:
import os, pickle
import numpy as np
import pandas as pd
from scipy import stats

FEATURES_BASE_PATH: str = "/kaggle/input/kc-swin-features"
MODELS_BASE_PATH: str = "/kaggle/input/kc-swin-features-train/models"
MODEL_NAMES: list = os.listdir(MODELS_BASE_PATH)
INT_TO_STR: dict = {
    0 : "cup",
    1 : "fork",
    2 : "glass",
    3 : "knife",
    4 : "plate",
    5 : "spoon"
}

In [2]:
ts_base_features  = np.load(f"{FEATURES_BASE_PATH}/ts_swin_base_patch4_window12_384_in22k.npy")
ts_large_features = np.load(f"{FEATURES_BASE_PATH}/ts_swin_large_patch4_window12_384_in22k.npy")

y_pred = np.zeros((1, ts_base_features.shape[0]), dtype=np.uint8)

for model_name in MODEL_NAMES:
    if "lgr" in model_name or "knc" in model_name or "svc" in model_name or "rfc" in model_name or "etcs" in model_name:
        if "base" in model_name:
            model = pickle.load(open(os.path.join(MODELS_BASE_PATH, model_name), "rb"))
            y_pred = np.concatenate((y_pred, model.predict(ts_base_features).reshape(1, -1)), axis=0)
        if "large" in model_name:
            model = pickle.load(open(os.path.join(MODELS_BASE_PATH, model_name), "rb"))
            y_pred = np.concatenate((y_pred, model.predict(ts_large_features).reshape(1, -1)), axis=0)

y_pred = stats.mode(y_pred[1:])[0].reshape(-1)
y_pred = [INT_TO_STR[int(sample)] for sample in y_pred]
                                    
ss_df = pd.read_csv("/kaggle/input/kitchenware-classification/sample_submission.csv")
ss_df.label = y_pred
ss_df.to_csv("submission.csv", index=False)